In [1]:
import pandas as pd
%matplotlib inline
import numpy as np
import pmdarima 
import re
import math
pd.options.mode.chained_assignment = None  # default='warn'

Caricamento dei dati e creazione del dataset.

In [2]:
mesi = ('Gennaio', 'Febbraio', 'Marzo', 'Aprile', 'Maggio', 'Giugno', 'Luglio', 'Agosto', 'Settembre', 'Ottobre', 
        'Novembre', 'Dicembre')

In [3]:
anni = ('2018', '2019', '2020')

In [4]:
csv_punto = ('Novembre2019', 'Agosto2020', 'Settembre2020', 
                         'Ottobre2020', 'Novembre2020', 'Dicembre2020')

In [5]:
csv_virgola = ('Dicembre2019', 'Aprile2020', 'Maggio2020')

In [6]:
diz = {}

In [7]:
for anno in anni:
    for mese in mesi:
        data = mese+anno
        if data == 'Agosto2018' or data == 'Aprile2019' :
            if data == 'Agosto2018' :
                Agosto1=pd.read_excel('Agosto2018(1-8).xlsx')
                Agosto2=pd.read_excel('Agosto2018(9-31).xlsx')
                diz['Agosto2018'] = pd.concat([Agosto1, Agosto2])
            else :
                Aprile1=pd.read_excel('Aprile2019(1-29).xlsx')
                Aprile2=pd.read_excel('Aprile2019(30).xlsx')
                diz['Aprile2019'] = pd.concat([Aprile1, Aprile2])
        else :
            if data in csv_virgola :
                formato = data+'.csv'
                diz[data] = pd.read_csv(formato, sep=";", decimal=",")
            if data in csv_punto :
                formato = data+'.csv'
                diz[data] = pd.read_csv(formato, sep=";", decimal=".")
            if (data not in csv_virgola) and (data not in csv_punto) :
                formato = data+'.xlsx'
                diz[data] = pd.read_excel(formato)          

In [8]:
u6 = diz['Gennaio2018']
for chiave in diz :
    if chiave != 'Gennaio2018':
        valore = diz[chiave]
        u6 = pd.concat([u6,valore])

In [9]:
u6['DATA'] = pd.to_datetime(u6['DATA'], format = '%Y%m%d')

In [10]:
u6.reset_index(inplace=True, drop=True)

In [11]:
u6['POD'].unique()
#IT012E00491824 POD per edificio U6
#IT012E00491869 POD per edificio U1

array(['IT012E00491824', 'IT012E00491869'], dtype=object)

In [12]:
u6[u6['POD'] == 'IT012E00491869']
#mese di giugno dell'edificio U1

POD       DATA     ORA  FL_ORA_LEGALE  \
84668  IT012E00491869 2020-06-01       0              2   
84669  IT012E00491869 2020-06-01    1500              2   
84670  IT012E00491869 2020-06-01    3000              2   
84671  IT012E00491869 2020-06-01    4500              2   
84672  IT012E00491869 2020-06-01   10000              2   
...               ...        ...     ...            ...   
87543  IT012E00491869 2020-06-30  224500              2   
87544  IT012E00491869 2020-06-30  230000              2   
87545  IT012E00491869 2020-06-30  231500              2   
87546  IT012E00491869 2020-06-30  233000              2   
87547  IT012E00491869 2020-06-30  234500              2   

       CONSUMO_ATTIVA_PRELEVATA  CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA  \
84668                      34.2                                  13.8   
84669                      42.0                                  19.8   
84670                      46.8                                  24.0   
84671                      43.2                                  20.4   
84672                      33.0                                  12.6   
...                         ...                                   ...   
87543                      91.8                                  39.0   
87544                      91.2                                  39.0   
87545                      91.2                                  37.2   
87546                      91.2                                  38.4   
87547                      90.6                                  37.2   

      POTENZA_MASSIMA TIPO_DATO Unnamed: 1 ORA.1  ...  Unnamed: 27  \
84668           405.6       NaN        NaN   NaN  ...          NaN   
84669           405.6       NaN        NaN   NaN  ...          NaN   
84670           405.6       NaN        NaN   NaN  ...          NaN   
84671           405.6       NaN        NaN   NaN  ...          NaN   
84672           405.6       NaN        NaN   NaN  ...          NaN   
...               ...       ...        ...   ...  ...          ...   
87543           405.6       NaN        NaN   NaN  ...          NaN   
87544           405.6       NaN        NaN   NaN  ...          NaN   
87545           405.6       NaN        NaN   NaN  ...          NaN   
87546           405.6       NaN        NaN   NaN  ...          NaN   
87547           405.6       NaN        NaN   NaN  ...          NaN   

       Unnamed: 28    anno  mese  giorno  giorno/sett   ora  Unnamed: 8  \
84668          NaN  2020.0   6.0     1.0   2020-06-01   1.0         NaN   
84669          NaN  2020.0   6.0     1.0   2020-06-01   1.0         NaN   
84670          NaN  2020.0   6.0     1.0   2020-06-01   1.0         NaN   
84671          NaN  2020.0   6.0     1.0   2020-06-01   1.0         NaN   
84672          NaN  2020.0   6.0     1.0   2020-06-01   2.0         NaN   
...            ...     ...   ...     ...          ...   ...         ...   
87543          NaN  2020.0   6.0    30.0   2020-06-30  23.0         NaN   
87544          NaN  2020.0   6.0    30.0   2020-06-30  24.0         NaN   
87545          NaN  2020.0   6.0    30.0   2020-06-30  24.0         NaN   
87546          NaN  2020.0   6.0    30.0   2020-06-30  24.0         NaN   
87547          NaN  2020.0   6.0    30.0   2020-06-30  24.0         NaN   

       Unnamed: 9  Unnamed: 10  
84668         NaN          NaN  
84669         NaN          NaN  
84670         NaN          NaN  
84671         NaN          NaN  
84672         NaN          NaN  
...           ...          ...  
87543         NaN          NaN  
87544         NaN          NaN  
87545         NaN          NaN  
87546         NaN          NaN  
87547         NaN          NaN  

[2880 rows x 37 columns]

Analisi e pulizia preliminare.

In [13]:
zeroconsumi = u6[u6['CONSUMO_ATTIVA_PRELEVATA'] == 0]
zeroconsumi.head()
len(zeroconsumi)

216

In [14]:
dateconzero = zeroconsumi['DATA'].unique()
for el in dateconzero:
    print(el)

2018-03-03T00:00:00.000000000
2018-12-27T00:00:00.000000000
2019-07-08T00:00:00.000000000
2020-04-11T00:00:00.000000000
2020-06-22T00:00:00.000000000
2020-07-01T00:00:00.000000000
2020-07-31T00:00:00.000000000
2020-10-25T00:00:00.000000000


In [15]:
u6[u6['DATA'] == '2018-03-03'].tail(60)
#valori anomali alle 15:15 e alle 16:00 con in mezzo gli zeri
#soluzione: ssostituire i valori anomali con il valore del quarto d'ora vicino mentre gli zeri con la media degli estremi

POD       DATA     ORA  FL_ORA_LEGALE  \
5892  IT012E00491824 2018-03-03   90000              1   
5893  IT012E00491824 2018-03-03   91500              1   
5894  IT012E00491824 2018-03-03   93000              1   
5895  IT012E00491824 2018-03-03   94500              1   
5896  IT012E00491824 2018-03-03  100000              1   
5897  IT012E00491824 2018-03-03  101500              1   
5898  IT012E00491824 2018-03-03  103000              1   
5899  IT012E00491824 2018-03-03  104500              1   
5900  IT012E00491824 2018-03-03  110000              1   
5901  IT012E00491824 2018-03-03  111500              1   
5902  IT012E00491824 2018-03-03  113000              1   
5903  IT012E00491824 2018-03-03  114500              1   
5904  IT012E00491824 2018-03-03  120000              1   
5905  IT012E00491824 2018-03-03  121500              1   
5906  IT012E00491824 2018-03-03  123000              1   
5907  IT012E00491824 2018-03-03  124500              1   
5908  IT012E00491824 2018-03-03  130000              1   
5909  IT012E00491824 2018-03-03  131500              1   
5910  IT012E00491824 2018-03-03  133000              1   
5911  IT012E00491824 2018-03-03  134500              1   
5912  IT012E00491824 2018-03-03  140000              1   
5913  IT012E00491824 2018-03-03  141500              1   
5914  IT012E00491824 2018-03-03  143000              1   
5915  IT012E00491824 2018-03-03  144500              1   
5916  IT012E00491824 2018-03-03  150000              1   
5917  IT012E00491824 2018-03-03  151500              1   
5918  IT012E00491824 2018-03-03  153000              1   
5919  IT012E00491824 2018-03-03  154500              1   
5920  IT012E00491824 2018-03-03  160000              1   
5921  IT012E00491824 2018-03-03  161500              1   
5922  IT012E00491824 2018-03-03  163000              1   
5923  IT012E00491824 2018-03-03  164500              1   
5924  IT012E00491824 2018-03-03  170000              1   
5925  IT012E00491824 2018-03-03  171500              1   
5926  IT012E00491824 2018-03-03  173000              1   
5927  IT012E00491824 2018-03-03  174500              1   
5928  IT012E00491824 2018-03-03  180000              1   
5929  IT012E00491824 2018-03-03  181500              1   
5930  IT012E00491824 2018-03-03  183000              1   
5931  IT012E00491824 2018-03-03  184500              1   
5932  IT012E00491824 2018-03-03  190000              1   
5933  IT012E00491824 2018-03-03  191500              1   
5934  IT012E00491824 2018-03-03  193000              1   
5935  IT012E00491824 2018-03-03  194500              1   
5936  IT012E00491824 2018-03-03  200000              1   
5937  IT012E00491824 2018-03-03  201500              1   
5938  IT012E00491824 2018-03-03  203000              1   
5939  IT012E00491824 2018-03-03  204500              1   
5940  IT012E00491824 2018-03-03  210000              1   
5941  IT012E00491824 2018-03-03  211500              1   
5942  IT012E00491824 2018-03-03  213000              1   
5943  IT012E00491824 2018-03-03  214500              1   
5944  IT012E00491824 2018-03-03  220000              1   
5945  IT012E00491824 2018-03-03  221500              1   
5946  IT012E00491824 2018-03-03  223000              1   
5947  IT012E00491824 2018-03-03  224500              1   
5948  IT012E00491824 2018-03-03  230000              1   
5949  IT012E00491824 2018-03-03  231500              1   
5950  IT012E00491824 2018-03-03  233000              1   
5951  IT012E00491824 2018-03-03  234500              1   

      CONSUMO_ATTIVA_PRELEVATA  CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA  \
5892                    111.10                                 47.85   
5893                    110.55                                 46.20   
5894                    110.00                                 44.00   
5895                    110.55                                 44.55   
5896                    110.55                                 44.00   
5897                    110.55                

In [16]:
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 151500) & (u6.FL_ORA_LEGALE == 1) & (u6.DATA == '2018-03-03')] = 108.90
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 160000) & (u6.FL_ORA_LEGALE == 1) & (u6.DATA == '2018-03-03')] = 95.70
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 153000) & (u6.FL_ORA_LEGALE == 1) & (u6.DATA == '2018-03-03')] = (108.90+95.70)/2
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 154500) & (u6.FL_ORA_LEGALE == 1) & (u6.DATA == '2018-03-03')] = (108.90+95.70)/2

In [17]:
u6[u6['DATA'] == '2018-12-27'].head(60)
#valore anomalo alle 09:30 mentre alle 12:00 lo consideriamo buono
#soluzione: sostituire il valore con la media dei valori dello stesso orario del giorno prima e del giorno dopo (26/12 e 28/12)

POD       DATA     ORA  FL_ORA_LEGALE  \
34560  IT012E00491824 2018-12-27       0              1   
34561  IT012E00491824 2018-12-27    1500              1   
34562  IT012E00491824 2018-12-27    3000              1   
34563  IT012E00491824 2018-12-27    4500              1   
34564  IT012E00491824 2018-12-27   10000              1   
34565  IT012E00491824 2018-12-27   11500              1   
34566  IT012E00491824 2018-12-27   13000              1   
34567  IT012E00491824 2018-12-27   14500              1   
34568  IT012E00491824 2018-12-27   20000              1   
34569  IT012E00491824 2018-12-27   21500              1   
34570  IT012E00491824 2018-12-27   23000              1   
34571  IT012E00491824 2018-12-27   24500              1   
34572  IT012E00491824 2018-12-27   30000              1   
34573  IT012E00491824 2018-12-27   31500              1   
34574  IT012E00491824 2018-12-27   33000              1   
34575  IT012E00491824 2018-12-27   34500              1   
34576  IT012E00491824 2018-12-27   40000              1   
34577  IT012E00491824 2018-12-27   41500              1   
34578  IT012E00491824 2018-12-27   43000              1   
34579  IT012E00491824 2018-12-27   44500              1   
34580  IT012E00491824 2018-12-27   50000              1   
34581  IT012E00491824 2018-12-27   51500              1   
34582  IT012E00491824 2018-12-27   53000              1   
34583  IT012E00491824 2018-12-27   54500              1   
34584  IT012E00491824 2018-12-27   60000              1   
34585  IT012E00491824 2018-12-27   61500              1   
34586  IT012E00491824 2018-12-27   63000              1   
34587  IT012E00491824 2018-12-27   64500              1   
34588  IT012E00491824 2018-12-27   70000              1   
34589  IT012E00491824 2018-12-27   71500              1   
34590  IT012E00491824 2018-12-27   73000              1   
34591  IT012E00491824 2018-12-27   74500              1   
34592  IT012E00491824 2018-12-27   80000              1   
34593  IT012E00491824 2018-12-27   81500              1   
34594  IT012E00491824 2018-12-27   83000              1   
34595  IT012E00491824 2018-12-27   84500              1   
34596  IT012E00491824 2018-12-27   90000              1   
34597  IT012E00491824 2018-12-27   91500              1   
34598  IT012E00491824 2018-12-27   93000              1   
34599  IT012E00491824 2018-12-27   94500              1   
34600  IT012E00491824 2018-12-27  100000              1   
34601  IT012E00491824 2018-12-27  101500              1   
34602  IT012E00491824 2018-12-27  103000              1   
34603  IT012E00491824 2018-12-27  104500              1   
34604  IT012E00491824 2018-12-27  110000              1   
34605  IT012E00491824 2018-12-27  111500              1   
34606  IT012E00491824 2018-12-27  113000              1   
34607  IT012E00491824 2018-12-27  114500              1   
34608  IT012E00491824 2018-12-27  120000              1   
34609  IT012E00491824 2018-12-27  121500              1   
34610  IT012E00491824 2018-12-27  123000              1   
34611  IT012E00491824 2018-12-27  124500              1   
34612  IT012E00491824 2018-12-27  130000              1   
34613  IT012E00491824 2018-12-27  131500              1   
34614  IT012E00491824 2018-12-27  133000              1   
34615  IT012E00491824 2018-12-27  134500              1   
34616  IT012E00491824 2018-12-27  140000              1   
34617  IT012E00491824 2018-12-27  141500              1   
34618  IT012E00491824 2018-12-27  143000              1   
34619  IT012E00491824 2018-12-27  144500              1   

       CONSUMO_ATTIVA_PRELEVATA  CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA  \
34560                     51.15                                 17.05   
34561                     51.70                                 17.05   
34562                     51.15                                 17.05   
34563                     51.70                                 17.60   
34564                     51.15                      

In [18]:
def pulizia(data_sistemare, prima_data, seconda_data, ora_inizio, ora_fine, edificio = u6, peso1 = 1, peso2 = 1):
    conta = ora_inizio
    while conta <= ora_fine :
        primo_valore = float(edificio[(edificio.ORA == conta) & (edificio.DATA == prima_data)]['CONSUMO_ATTIVA_PRELEVATA'])
        secondo_valore = float(edificio[(edificio.ORA == conta) & (edificio.DATA == seconda_data)]['CONSUMO_ATTIVA_PRELEVATA'])
        media = round(((primo_valore*peso1) + (secondo_valore*peso2)) / (peso1+peso2),2)
        edificio['CONSUMO_ATTIVA_PRELEVATA'].loc[(edificio.ORA == conta) & (edificio.DATA == data_sistemare)] = media
        if '45' in str(conta):
            conta += 5500
        else:
            conta += 1500

In [19]:
pulizia('2018-12-27', '2018-12-26', '2018-12-28', 93000, 114500)

In [20]:
u6[u6['DATA'] == '2019-07-08'].tail(60)
#gli zeri vanno dalle 21:45 alle 23:45 (il valore alle 21:30 lo consideriamo buono)
#soluzione: sostituire gli zeri con la media pesata 0.8 del giorno successivo (stessi orari) e dello stesso giorno della 
#settimana prima con peso 0.2 (09-07 e 01-07)

POD       DATA     ORA  FL_ORA_LEGALE  \
53120  IT012E00491824 2019-07-08   90000              2   
53121  IT012E00491824 2019-07-08   91500              2   
53122  IT012E00491824 2019-07-08   93000              2   
53123  IT012E00491824 2019-07-08   94500              2   
53124  IT012E00491824 2019-07-08  100000              2   
53125  IT012E00491824 2019-07-08  101500              2   
53126  IT012E00491824 2019-07-08  103000              2   
53127  IT012E00491824 2019-07-08  104500              2   
53128  IT012E00491824 2019-07-08  110000              2   
53129  IT012E00491824 2019-07-08  111500              2   
53130  IT012E00491824 2019-07-08  113000              2   
53131  IT012E00491824 2019-07-08  114500              2   
53132  IT012E00491824 2019-07-08  120000              2   
53133  IT012E00491824 2019-07-08  121500              2   
53134  IT012E00491824 2019-07-08  123000              2   
53135  IT012E00491824 2019-07-08  124500              2   
53136  IT012E00491824 2019-07-08  130000              2   
53137  IT012E00491824 2019-07-08  131500              2   
53138  IT012E00491824 2019-07-08  133000              2   
53139  IT012E00491824 2019-07-08  134500              2   
53140  IT012E00491824 2019-07-08  140000              2   
53141  IT012E00491824 2019-07-08  141500              2   
53142  IT012E00491824 2019-07-08  143000              2   
53143  IT012E00491824 2019-07-08  144500              2   
53144  IT012E00491824 2019-07-08  150000              2   
53145  IT012E00491824 2019-07-08  151500              2   
53146  IT012E00491824 2019-07-08  153000              2   
53147  IT012E00491824 2019-07-08  154500              2   
53148  IT012E00491824 2019-07-08  160000              2   
53149  IT012E00491824 2019-07-08  161500              2   
53150  IT012E00491824 2019-07-08  163000              2   
53151  IT012E00491824 2019-07-08  164500              2   
53152  IT012E00491824 2019-07-08  170000              2   
53153  IT012E00491824 2019-07-08  171500              2   
53154  IT012E00491824 2019-07-08  173000              2   
53155  IT012E00491824 2019-07-08  174500              2   
53156  IT012E00491824 2019-07-08  180000              2   
53157  IT012E00491824 2019-07-08  181500              2   
53158  IT012E00491824 2019-07-08  183000              2   
53159  IT012E00491824 2019-07-08  184500              2   
53160  IT012E00491824 2019-07-08  190000              2   
53161  IT012E00491824 2019-07-08  191500              2   
53162  IT012E00491824 2019-07-08  193000              2   
53163  IT012E00491824 2019-07-08  194500              2   
53164  IT012E00491824 2019-07-08  200000              2   
53165  IT012E00491824 2019-07-08  201500              2   
53166  IT012E00491824 2019-07-08  203000              2   
53167  IT012E00491824 2019-07-08  204500              2   
53168  IT012E00491824 2019-07-08  210000              2   
53169  IT012E00491824 2019-07-08  211500              2   
53170  IT012E00491824 2019-07-08  213000              2   
53171  IT012E00491824 2019-07-08  214500              2   
53172  IT012E00491824 2019-07-08  220000              2   
53173  IT012E00491824 2019-07-08  221500              2   
53174  IT012E00491824 2019-07-08  223000              2   
53175  IT012E00491824 2019-07-08  224500              2   
53176  IT012E00491824 2019-07-08  230000              2   
53177  IT012E00491824 2019-07-08  231500              2   
53178  IT012E00491824 2019-07-08  233000              2   
53179  IT012E00491824 2019-07-08  234500              2   

       CONSUMO_ATTIVA_PRELEVATA  CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA  \
53120                    191.40                                 84.15   
53121                    193.60                                 85.80   
53122                    193.60                                 89.65   
53123                    196.90                                 90.20   
53124                    199.65                      

In [21]:
pulizia('2019-07-08', '2019-07-09', '2019-07-01', 214500, 234500, peso1 = 0.8, peso2 = 0.2)

In [22]:
u6[u6['DATA'] == '2019-07-09'].head(60)
#valori anomali dalle 00:00 e alle 07:45 (alle 08:00 lo consideriamo buono)
#soluzione: sostituire i valori anomali con il valori pesati del giorno dopo e dello stesso giorno della settimana prima
#con pesi 0.8 e 0.2 (10-07 e 02-07)

POD       DATA     ORA  FL_ORA_LEGALE  \
53180  IT012E00491824 2019-07-09       0              2   
53181  IT012E00491824 2019-07-09    1500              2   
53182  IT012E00491824 2019-07-09    3000              2   
53183  IT012E00491824 2019-07-09    4500              2   
53184  IT012E00491824 2019-07-09   10000              2   
53185  IT012E00491824 2019-07-09   11500              2   
53186  IT012E00491824 2019-07-09   13000              2   
53187  IT012E00491824 2019-07-09   14500              2   
53188  IT012E00491824 2019-07-09   20000              2   
53189  IT012E00491824 2019-07-09   21500              2   
53190  IT012E00491824 2019-07-09   23000              2   
53191  IT012E00491824 2019-07-09   24500              2   
53192  IT012E00491824 2019-07-09   30000              2   
53193  IT012E00491824 2019-07-09   31500              2   
53194  IT012E00491824 2019-07-09   33000              2   
53195  IT012E00491824 2019-07-09   34500              2   
53196  IT012E00491824 2019-07-09   40000              2   
53197  IT012E00491824 2019-07-09   41500              2   
53198  IT012E00491824 2019-07-09   43000              2   
53199  IT012E00491824 2019-07-09   44500              2   
53200  IT012E00491824 2019-07-09   50000              2   
53201  IT012E00491824 2019-07-09   51500              2   
53202  IT012E00491824 2019-07-09   53000              2   
53203  IT012E00491824 2019-07-09   54500              2   
53204  IT012E00491824 2019-07-09   60000              2   
53205  IT012E00491824 2019-07-09   61500              2   
53206  IT012E00491824 2019-07-09   63000              2   
53207  IT012E00491824 2019-07-09   64500              2   
53208  IT012E00491824 2019-07-09   70000              2   
53209  IT012E00491824 2019-07-09   71500              2   
53210  IT012E00491824 2019-07-09   73000              2   
53211  IT012E00491824 2019-07-09   74500              2   
53212  IT012E00491824 2019-07-09   80000              2   
53213  IT012E00491824 2019-07-09   81500              2   
53214  IT012E00491824 2019-07-09   83000              2   
53215  IT012E00491824 2019-07-09   84500              2   
53216  IT012E00491824 2019-07-09   90000              2   
53217  IT012E00491824 2019-07-09   91500              2   
53218  IT012E00491824 2019-07-09   93000              2   
53219  IT012E00491824 2019-07-09   94500              2   
53220  IT012E00491824 2019-07-09  100000              2   
53221  IT012E00491824 2019-07-09  101500              2   
53222  IT012E00491824 2019-07-09  103000              2   
53223  IT012E00491824 2019-07-09  104500              2   
53224  IT012E00491824 2019-07-09  110000              2   
53225  IT012E00491824 2019-07-09  111500              2   
53226  IT012E00491824 2019-07-09  113000              2   
53227  IT012E00491824 2019-07-09  114500              2   
53228  IT012E00491824 2019-07-09  120000              2   
53229  IT012E00491824 2019-07-09  121500              2   
53230  IT012E00491824 2019-07-09  123000              2   
53231  IT012E00491824 2019-07-09  124500              2   
53232  IT012E00491824 2019-07-09  130000              2   
53233  IT012E00491824 2019-07-09  131500              2   
53234  IT012E00491824 2019-07-09  133000              2   
53235  IT012E00491824 2019-07-09  134500              2   
53236  IT012E00491824 2019-07-09  140000              2   
53237  IT012E00491824 2019-07-09  141500              2   
53238  IT012E00491824 2019-07-09  143000              2   
53239  IT012E00491824 2019-07-09  144500              2   

       CONSUMO_ATTIVA_PRELEVATA  CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA  \
53180                     36.85                                 25.30   
53181                     36.30                                 24.75   
53182                     34.65                                 23.10   
53183                     35.20                                 22.55   
53184                     34.65                      

In [23]:
pulizia('2019-07-09', '2019-07-10', '2019-07-02', 0, 74500, peso1 = 0.8, peso2 = 0.2)

In [24]:
u6[u6['DATA'] == '2020-04-11'].head(60)
#valori anomali alle 08:30 e alle 14:00 con in mezzo gli zeri
#soluzione: sostituire i valori anomali con con i valori pesati del giorno successivo e stesso giorno 
#della settimana prima con pesi 0.8 e 0.2 (conformità dei consumi durante la pandemia)

POD       DATA     ORA  FL_ORA_LEGALE  \
79772  IT012E00491824 2020-04-11       0              2   
79773  IT012E00491824 2020-04-11    1500              2   
79774  IT012E00491824 2020-04-11    3000              2   
79775  IT012E00491824 2020-04-11    4500              2   
79776  IT012E00491824 2020-04-11   10000              2   
79777  IT012E00491824 2020-04-11   11500              2   
79778  IT012E00491824 2020-04-11   13000              2   
79779  IT012E00491824 2020-04-11   14500              2   
79780  IT012E00491824 2020-04-11   20000              2   
79781  IT012E00491824 2020-04-11   21500              2   
79782  IT012E00491824 2020-04-11   23000              2   
79783  IT012E00491824 2020-04-11   24500              2   
79784  IT012E00491824 2020-04-11   30000              2   
79785  IT012E00491824 2020-04-11   31500              2   
79786  IT012E00491824 2020-04-11   33000              2   
79787  IT012E00491824 2020-04-11   34500              2   
79788  IT012E00491824 2020-04-11   40000              2   
79789  IT012E00491824 2020-04-11   41500              2   
79790  IT012E00491824 2020-04-11   43000              2   
79791  IT012E00491824 2020-04-11   44500              2   
79792  IT012E00491824 2020-04-11   50000              2   
79793  IT012E00491824 2020-04-11   51500              2   
79794  IT012E00491824 2020-04-11   53000              2   
79795  IT012E00491824 2020-04-11   54500              2   
79796  IT012E00491824 2020-04-11   60000              2   
79797  IT012E00491824 2020-04-11   61500              2   
79798  IT012E00491824 2020-04-11   63000              2   
79799  IT012E00491824 2020-04-11   64500              2   
79800  IT012E00491824 2020-04-11   70000              2   
79801  IT012E00491824 2020-04-11   71500              2   
79802  IT012E00491824 2020-04-11   73000              2   
79803  IT012E00491824 2020-04-11   74500              2   
79804  IT012E00491824 2020-04-11   80000              2   
79805  IT012E00491824 2020-04-11   81500              2   
79806  IT012E00491824 2020-04-11   83000              2   
79807  IT012E00491824 2020-04-11   84500              2   
79808  IT012E00491824 2020-04-11   90000              2   
79809  IT012E00491824 2020-04-11   91500              2   
79810  IT012E00491824 2020-04-11   93000              2   
79811  IT012E00491824 2020-04-11   94500              2   
79812  IT012E00491824 2020-04-11  100000              2   
79813  IT012E00491824 2020-04-11  101500              2   
79814  IT012E00491824 2020-04-11  103000              2   
79815  IT012E00491824 2020-04-11  104500              2   
79816  IT012E00491824 2020-04-11  110000              2   
79817  IT012E00491824 2020-04-11  111500              2   
79818  IT012E00491824 2020-04-11  113000              2   
79819  IT012E00491824 2020-04-11  114500              2   
79820  IT012E00491824 2020-04-11  120000              2   
79821  IT012E00491824 2020-04-11  121500              2   
79822  IT012E00491824 2020-04-11  123000              2   
79823  IT012E00491824 2020-04-11  124500              2   
79824  IT012E00491824 2020-04-11  130000              2   
79825  IT012E00491824 2020-04-11  131500              2   
79826  IT012E00491824 2020-04-11  133000              2   
79827  IT012E00491824 2020-04-11  134500              2   
79828  IT012E00491824 2020-04-11  140000              2   
79829  IT012E00491824 2020-04-11  141500              2   
79830  IT012E00491824 2020-04-11  143000              2   
79831  IT012E00491824 2020-04-11  144500              2   

       CONSUMO_ATTIVA_PRELEVATA  CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA  \
79772                     78.10                                 20.35   
79773                     77.55                                 20.35   
79774                     76.45                                 19.25   
79775                     75.90                                 18.15   
79776                     74.80                      

In [25]:
pulizia('2020-04-11', '2020-04-12', '2020-04-04', 83000, 140000, peso1 = 0.8, peso2 = 0.2)

In [26]:
u6[u6['DATA'] == '2020-06-22'].head(60)
#valore zero alle 12:45 e valore anomali alle 13:00
#soluzione: sostituire con i valori degli estremi

POD       DATA     ORA  FL_ORA_LEGALE  \
86684  IT012E00491869 2020-06-22       0              2   
86685  IT012E00491869 2020-06-22    1500              2   
86686  IT012E00491869 2020-06-22    3000              2   
86687  IT012E00491869 2020-06-22    4500              2   
86688  IT012E00491869 2020-06-22   10000              2   
86689  IT012E00491869 2020-06-22   11500              2   
86690  IT012E00491869 2020-06-22   13000              2   
86691  IT012E00491869 2020-06-22   14500              2   
86692  IT012E00491869 2020-06-22   20000              2   
86693  IT012E00491869 2020-06-22   21500              2   
86694  IT012E00491869 2020-06-22   23000              2   
86695  IT012E00491869 2020-06-22   24500              2   
86696  IT012E00491869 2020-06-22   30000              2   
86697  IT012E00491869 2020-06-22   31500              2   
86698  IT012E00491869 2020-06-22   33000              2   
86699  IT012E00491869 2020-06-22   34500              2   
86700  IT012E00491869 2020-06-22   40000              2   
86701  IT012E00491869 2020-06-22   41500              2   
86702  IT012E00491869 2020-06-22   43000              2   
86703  IT012E00491869 2020-06-22   44500              2   
86704  IT012E00491869 2020-06-22   50000              2   
86705  IT012E00491869 2020-06-22   51500              2   
86706  IT012E00491869 2020-06-22   53000              2   
86707  IT012E00491869 2020-06-22   54500              2   
86708  IT012E00491869 2020-06-22   60000              2   
86709  IT012E00491869 2020-06-22   61500              2   
86710  IT012E00491869 2020-06-22   63000              2   
86711  IT012E00491869 2020-06-22   64500              2   
86712  IT012E00491869 2020-06-22   70000              2   
86713  IT012E00491869 2020-06-22   71500              2   
86714  IT012E00491869 2020-06-22   73000              2   
86715  IT012E00491869 2020-06-22   74500              2   
86716  IT012E00491869 2020-06-22   80000              2   
86717  IT012E00491869 2020-06-22   81500              2   
86718  IT012E00491869 2020-06-22   83000              2   
86719  IT012E00491869 2020-06-22   84500              2   
86720  IT012E00491869 2020-06-22   90000              2   
86721  IT012E00491869 2020-06-22   91500              2   
86722  IT012E00491869 2020-06-22   93000              2   
86723  IT012E00491869 2020-06-22   94500              2   
86724  IT012E00491869 2020-06-22  100000              2   
86725  IT012E00491869 2020-06-22  101500              2   
86726  IT012E00491869 2020-06-22  103000              2   
86727  IT012E00491869 2020-06-22  104500              2   
86728  IT012E00491869 2020-06-22  110000              2   
86729  IT012E00491869 2020-06-22  111500              2   
86730  IT012E00491869 2020-06-22  113000              2   
86731  IT012E00491869 2020-06-22  114500              2   
86732  IT012E00491869 2020-06-22  120000              2   
86733  IT012E00491869 2020-06-22  121500              2   
86734  IT012E00491869 2020-06-22  123000              2   
86735  IT012E00491869 2020-06-22  124500              2   
86736  IT012E00491869 2020-06-22  130000              2   
86737  IT012E00491869 2020-06-22  131500              2   
86738  IT012E00491869 2020-06-22  133000              2   
86739  IT012E00491869 2020-06-22  134500              2   
86740  IT012E00491869 2020-06-22  140000              2   
86741  IT012E00491869 2020-06-22  141500              2   
86742  IT012E00491869 2020-06-22  143000              2   
86743  IT012E00491869 2020-06-22  144500              2   

       CONSUMO_ATTIVA_PRELEVATA  CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA  \
86684                      34.8                                   7.2   
86685                      34.8                                   7.2   
86686                      35.4                                   7.8   
86687                      34.8                                   7.2   
86688                      34.8                      

In [27]:
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 124500) & (u6.DATA == '2020-06-22')] = 79.2
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 130000) & (u6.DATA == '2020-06-22')] = 68.4

In [28]:
u6[u6['DATA'] == '2020-07-01'].tail(60)
#valori anomali alle 19:15 e alle 19:45 con lo zero alle 19:30
#soluzione: sostituire gli estremi e fare la media per sostituire lo zero

POD       DATA     ORA  FL_ORA_LEGALE  \
87584  IT012E00491824 2020-07-01   90000              2   
87585  IT012E00491824 2020-07-01   91500              2   
87586  IT012E00491824 2020-07-01   93000              2   
87587  IT012E00491824 2020-07-01   94500              2   
87588  IT012E00491824 2020-07-01  100000              2   
87589  IT012E00491824 2020-07-01  101500              2   
87590  IT012E00491824 2020-07-01  103000              2   
87591  IT012E00491824 2020-07-01  104500              2   
87592  IT012E00491824 2020-07-01  110000              2   
87593  IT012E00491824 2020-07-01  111500              2   
87594  IT012E00491824 2020-07-01  113000              2   
87595  IT012E00491824 2020-07-01  114500              2   
87596  IT012E00491824 2020-07-01  120000              2   
87597  IT012E00491824 2020-07-01  121500              2   
87598  IT012E00491824 2020-07-01  123000              2   
87599  IT012E00491824 2020-07-01  124500              2   
87600  IT012E00491824 2020-07-01  130000              2   
87601  IT012E00491824 2020-07-01  131500              2   
87602  IT012E00491824 2020-07-01  133000              2   
87603  IT012E00491824 2020-07-01  134500              2   
87604  IT012E00491824 2020-07-01  140000              2   
87605  IT012E00491824 2020-07-01  141500              2   
87606  IT012E00491824 2020-07-01  143000              2   
87607  IT012E00491824 2020-07-01  144500              2   
87608  IT012E00491824 2020-07-01  150000              2   
87609  IT012E00491824 2020-07-01  151500              2   
87610  IT012E00491824 2020-07-01  153000              2   
87611  IT012E00491824 2020-07-01  154500              2   
87612  IT012E00491824 2020-07-01  160000              2   
87613  IT012E00491824 2020-07-01  161500              2   
87614  IT012E00491824 2020-07-01  163000              2   
87615  IT012E00491824 2020-07-01  164500              2   
87616  IT012E00491824 2020-07-01  170000              2   
87617  IT012E00491824 2020-07-01  171500              2   
87618  IT012E00491824 2020-07-01  173000              2   
87619  IT012E00491824 2020-07-01  174500              2   
87620  IT012E00491824 2020-07-01  180000              2   
87621  IT012E00491824 2020-07-01  181500              2   
87622  IT012E00491824 2020-07-01  183000              2   
87623  IT012E00491824 2020-07-01  184500              2   
87624  IT012E00491824 2020-07-01  190000              2   
87625  IT012E00491824 2020-07-01  191500              2   
87626  IT012E00491824 2020-07-01  193000              2   
87627  IT012E00491824 2020-07-01  194500              2   
87628  IT012E00491824 2020-07-01  200000              2   
87629  IT012E00491824 2020-07-01  201500              2   
87630  IT012E00491824 2020-07-01  203000              2   
87631  IT012E00491824 2020-07-01  204500              2   
87632  IT012E00491824 2020-07-01  210000              2   
87633  IT012E00491824 2020-07-01  211500              2   
87634  IT012E00491824 2020-07-01  213000              2   
87635  IT012E00491824 2020-07-01  214500              2   
87636  IT012E00491824 2020-07-01  220000              2   
87637  IT012E00491824 2020-07-01  221500              2   
87638  IT012E00491824 2020-07-01  223000              2   
87639  IT012E00491824 2020-07-01  224500              2   
87640  IT012E00491824 2020-07-01  230000              2   
87641  IT012E00491824 2020-07-01  231500              2   
87642  IT012E00491824 2020-07-01  233000              2   
87643  IT012E00491824 2020-07-01  234500              2   

       CONSUMO_ATTIVA_PRELEVATA  CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA  \
87584                    129.80                                 53.35   
87585                    128.70                                 52.25   
87586                    130.35                                 52.80   
87587                    132.55                                 53.90   
87588                    132.55                      

In [29]:
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 191500) & (u6.DATA == '2020-07-01')] = 125.40
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 194500) & (u6.DATA == '2020-07-01')] = 84.70
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 193000) & (u6.DATA == '2020-07-01')] = (125.40+84.70)/2

In [30]:
u6[u6['DATA'] == '2020-07-31'].head(60)
u6[u6['DATA'] == '2020-07-31'].tail(60)
#gli zeri su tutto il giorno
#soluzione: valori pesati del giorno dopo e dello stesso giorno della settimana dopo con pesi pari a 0.8 e 0.2

POD       DATA     ORA  FL_ORA_LEGALE  \
90464  IT012E00491824 2020-07-31   90000              2   
90465  IT012E00491824 2020-07-31   91500              2   
90466  IT012E00491824 2020-07-31   93000              2   
90467  IT012E00491824 2020-07-31   94500              2   
90468  IT012E00491824 2020-07-31  100000              2   
90469  IT012E00491824 2020-07-31  101500              2   
90470  IT012E00491824 2020-07-31  103000              2   
90471  IT012E00491824 2020-07-31  104500              2   
90472  IT012E00491824 2020-07-31  110000              2   
90473  IT012E00491824 2020-07-31  111500              2   
90474  IT012E00491824 2020-07-31  113000              2   
90475  IT012E00491824 2020-07-31  114500              2   
90476  IT012E00491824 2020-07-31  120000              2   
90477  IT012E00491824 2020-07-31  121500              2   
90478  IT012E00491824 2020-07-31  123000              2   
90479  IT012E00491824 2020-07-31  124500              2   
90480  IT012E00491824 2020-07-31  130000              2   
90481  IT012E00491824 2020-07-31  131500              2   
90482  IT012E00491824 2020-07-31  133000              2   
90483  IT012E00491824 2020-07-31  134500              2   
90484  IT012E00491824 2020-07-31  140000              2   
90485  IT012E00491824 2020-07-31  141500              2   
90486  IT012E00491824 2020-07-31  143000              2   
90487  IT012E00491824 2020-07-31  144500              2   
90488  IT012E00491824 2020-07-31  150000              2   
90489  IT012E00491824 2020-07-31  151500              2   
90490  IT012E00491824 2020-07-31  153000              2   
90491  IT012E00491824 2020-07-31  154500              2   
90492  IT012E00491824 2020-07-31  160000              2   
90493  IT012E00491824 2020-07-31  161500              2   
90494  IT012E00491824 2020-07-31  163000              2   
90495  IT012E00491824 2020-07-31  164500              2   
90496  IT012E00491824 2020-07-31  170000              2   
90497  IT012E00491824 2020-07-31  171500              2   
90498  IT012E00491824 2020-07-31  173000              2   
90499  IT012E00491824 2020-07-31  174500              2   
90500  IT012E00491824 2020-07-31  180000              2   
90501  IT012E00491824 2020-07-31  181500              2   
90502  IT012E00491824 2020-07-31  183000              2   
90503  IT012E00491824 2020-07-31  184500              2   
90504  IT012E00491824 2020-07-31  190000              2   
90505  IT012E00491824 2020-07-31  191500              2   
90506  IT012E00491824 2020-07-31  193000              2   
90507  IT012E00491824 2020-07-31  194500              2   
90508  IT012E00491824 2020-07-31  200000              2   
90509  IT012E00491824 2020-07-31  201500              2   
90510  IT012E00491824 2020-07-31  203000              2   
90511  IT012E00491824 2020-07-31  204500              2   
90512  IT012E00491824 2020-07-31  210000              2   
90513  IT012E00491824 2020-07-31  211500              2   
90514  IT012E00491824 2020-07-31  213000              2   
90515  IT012E00491824 2020-07-31  214500              2   
90516  IT012E00491824 2020-07-31  220000              2   
90517  IT012E00491824 2020-07-31  221500              2   
90518  IT012E00491824 2020-07-31  223000              2   
90519  IT012E00491824 2020-07-31  224500              2   
90520  IT012E00491824 2020-07-31  230000              2   
90521  IT012E00491824 2020-07-31  231500              2   
90522  IT012E00491824 2020-07-31  233000              2   
90523  IT012E00491824 2020-07-31  234500              2   

       CONSUMO_ATTIVA_PRELEVATA  CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA  \
90464                       0.0                                   0.0   
90465                       0.0                                   0.0   
90466                       0.0                                   0.0   
90467                       0.0                                   0.0   
90468                       0.0                      

In [31]:
pulizia('2020-07-31', '2020-08-01', '2020-08-07', 0, 234500, peso1 = 0.8, peso2 = 0.2)

In [32]:
#25-10-2018 risolto nei passaggi successivi

In [33]:
giorni = u6['DATA'].unique()
for giorno in giorni :
    dati_del_giorno = u6[u6['DATA'] == giorno]
    if len(dati_del_giorno) != 96:
        print(giorno, len(dati_del_giorno))

2018-03-25T00:00:00.000000000 92
2018-10-28T00:00:00.000000000 100
2019-03-31T00:00:00.000000000 92
2019-10-27T00:00:00.000000000 100
2020-03-29T00:00:00.000000000 92
2020-10-25T00:00:00.000000000 192


In [34]:
u6[u6['DATA'] == '2020-10-25'].head(60)
#gli zeri iniziano alle ore 3:00 con l'ultimo valore osservato alle ore 2:45 con consumo pari a 96.25

POD       DATA     ORA  FL_ORA_LEGALE  \
98684  IT012E00491824 2020-10-25       0              2   
98685  IT012E00491824 2020-10-25    1500              2   
98686  IT012E00491824 2020-10-25    3000              2   
98687  IT012E00491824 2020-10-25    4500              2   
98688  IT012E00491824 2020-10-25   10000              2   
98689  IT012E00491824 2020-10-25   11500              2   
98690  IT012E00491824 2020-10-25   13000              2   
98691  IT012E00491824 2020-10-25   14500              2   
98692  IT012E00491824 2020-10-25   20000              2   
98693  IT012E00491824 2020-10-25   21500              2   
98694  IT012E00491824 2020-10-25   23000              2   
98695  IT012E00491824 2020-10-25   24500              2   
98696  IT012E00491824 2020-10-25   30000              2   
98697  IT012E00491824 2020-10-25   31500              2   
98698  IT012E00491824 2020-10-25   33000              2   
98699  IT012E00491824 2020-10-25   34500              2   
98700  IT012E00491824 2020-10-25   40000              2   
98701  IT012E00491824 2020-10-25   41500              2   
98702  IT012E00491824 2020-10-25   43000              2   
98703  IT012E00491824 2020-10-25   44500              2   
98704  IT012E00491824 2020-10-25   50000              2   
98705  IT012E00491824 2020-10-25   51500              2   
98706  IT012E00491824 2020-10-25   53000              2   
98707  IT012E00491824 2020-10-25   54500              2   
98708  IT012E00491824 2020-10-25   60000              2   
98709  IT012E00491824 2020-10-25   61500              2   
98710  IT012E00491824 2020-10-25   63000              2   
98711  IT012E00491824 2020-10-25   64500              2   
98712  IT012E00491824 2020-10-25   70000              2   
98713  IT012E00491824 2020-10-25   71500              2   
98714  IT012E00491824 2020-10-25   73000              2   
98715  IT012E00491824 2020-10-25   74500              2   
98716  IT012E00491824 2020-10-25   80000              2   
98717  IT012E00491824 2020-10-25   81500              2   
98718  IT012E00491824 2020-10-25   83000              2   
98719  IT012E00491824 2020-10-25   84500              2   
98720  IT012E00491824 2020-10-25   90000              2   
98721  IT012E00491824 2020-10-25   91500              2   
98722  IT012E00491824 2020-10-25   93000              2   
98723  IT012E00491824 2020-10-25   94500              2   
98724  IT012E00491824 2020-10-25  100000              2   
98725  IT012E00491824 2020-10-25  101500              2   
98726  IT012E00491824 2020-10-25  103000              2   
98727  IT012E00491824 2020-10-25  104500              2   
98728  IT012E00491824 2020-10-25  110000              2   
98729  IT012E00491824 2020-10-25  111500              2   
98730  IT012E00491824 2020-10-25  113000              2   
98731  IT012E00491824 2020-10-25  114500              2   
98732  IT012E00491824 2020-10-25  120000              2   
98733  IT012E00491824 2020-10-25  121500              2   
98734  IT012E00491824 2020-10-25  123000              2   
98735  IT012E00491824 2020-10-25  124500              2   
98736  IT012E00491824 2020-10-25  130000              2   
98737  IT012E00491824 2020-10-25  131500              2   
98738  IT012E00491824 2020-10-25  133000              2   
98739  IT012E00491824 2020-10-25  134500              2   
98740  IT012E00491824 2020-10-25  140000              2   
98741  IT012E00491824 2020-10-25  141500              2   
98742  IT012E00491824 2020-10-25  143000              2   
98743  IT012E00491824 2020-10-25  144500              2   

       CONSUMO_ATTIVA_PRELEVATA  CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA  \
98684                     97.35                                 34.10   
98685                     96.80                                 33.55   
98686                     97.35                                 34.10   
98687                     97.35                                 34.65   
98688                     97.35                      

In [35]:
u6[(u6['DATA'] == '2020-10-25') & (u6['FL_ORA_LEGALE'] == 1)].head(60)

POD       DATA     ORA  FL_ORA_LEGALE  \
98780  IT012E00491824 2020-10-25       0              1   
98781  IT012E00491824 2020-10-25    1500              1   
98782  IT012E00491824 2020-10-25    3000              1   
98783  IT012E00491824 2020-10-25    4500              1   
98784  IT012E00491824 2020-10-25   10000              1   
98785  IT012E00491824 2020-10-25   11500              1   
98786  IT012E00491824 2020-10-25   13000              1   
98787  IT012E00491824 2020-10-25   14500              1   
98788  IT012E00491824 2020-10-25   20000              1   
98789  IT012E00491824 2020-10-25   21500              1   
98790  IT012E00491824 2020-10-25   23000              1   
98791  IT012E00491824 2020-10-25   24500              1   
98792  IT012E00491824 2020-10-25   30000              1   
98793  IT012E00491824 2020-10-25   31500              1   
98794  IT012E00491824 2020-10-25   33000              1   
98795  IT012E00491824 2020-10-25   34500              1   
98796  IT012E00491824 2020-10-25   40000              1   
98797  IT012E00491824 2020-10-25   41500              1   
98798  IT012E00491824 2020-10-25   43000              1   
98799  IT012E00491824 2020-10-25   44500              1   
98800  IT012E00491824 2020-10-25   50000              1   
98801  IT012E00491824 2020-10-25   51500              1   
98802  IT012E00491824 2020-10-25   53000              1   
98803  IT012E00491824 2020-10-25   54500              1   
98804  IT012E00491824 2020-10-25   60000              1   
98805  IT012E00491824 2020-10-25   61500              1   
98806  IT012E00491824 2020-10-25   63000              1   
98807  IT012E00491824 2020-10-25   64500              1   
98808  IT012E00491824 2020-10-25   70000              1   
98809  IT012E00491824 2020-10-25   71500              1   
98810  IT012E00491824 2020-10-25   73000              1   
98811  IT012E00491824 2020-10-25   74500              1   
98812  IT012E00491824 2020-10-25   80000              1   
98813  IT012E00491824 2020-10-25   81500              1   
98814  IT012E00491824 2020-10-25   83000              1   
98815  IT012E00491824 2020-10-25   84500              1   
98816  IT012E00491824 2020-10-25   90000              1   
98817  IT012E00491824 2020-10-25   91500              1   
98818  IT012E00491824 2020-10-25   93000              1   
98819  IT012E00491824 2020-10-25   94500              1   
98820  IT012E00491824 2020-10-25  100000              1   
98821  IT012E00491824 2020-10-25  101500              1   
98822  IT012E00491824 2020-10-25  103000              1   
98823  IT012E00491824 2020-10-25  104500              1   
98824  IT012E00491824 2020-10-25  110000              1   
98825  IT012E00491824 2020-10-25  111500              1   
98826  IT012E00491824 2020-10-25  113000              1   
98827  IT012E00491824 2020-10-25  114500              1   
98828  IT012E00491824 2020-10-25  120000              1   
98829  IT012E00491824 2020-10-25  121500              1   
98830  IT012E00491824 2020-10-25  123000              1   
98831  IT012E00491824 2020-10-25  124500              1   
98832  IT012E00491824 2020-10-25  130000              1   
98833  IT012E00491824 2020-10-25  131500              1   
98834  IT012E00491824 2020-10-25  133000              1   
98835  IT012E00491824 2020-10-25  134500              1   
98836  IT012E00491824 2020-10-25  140000              1   
98837  IT012E00491824 2020-10-25  141500              1   
98838  IT012E00491824 2020-10-25  143000              1   
98839  IT012E00491824 2020-10-25  144500              1   

       CONSUMO_ATTIVA_PRELEVATA  CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA  \
98780                      0.00                                  0.00   
98781                      0.00                                  0.00   
98782                      0.00                                  0.00   
98783                      0.00                                  0.00   
98784                      0.00                      

In [36]:
#C'è una certa corrispondeza tra i valori: nel dataset di prima (quello con i valori nulli) 
#avevamo alle ore 2:30 un consumo di #95.70 kw mentre alle ore 2:45 pari a 96.25. 
#Nel secondo dataset abbiamo gli stessi valori ma rispettivamente alle ore 2:45 e 3:00. 
#Dunque conviene tenere come buona la seconda tabella e sostituire i pochi zeri che ci sono 
#con i valori della tabella precedente.

In [37]:
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 0) & (u6.FL_ORA_LEGALE == 1) & (u6.DATA == '2020-10-25')] = 97.35
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 1500) & (u6.FL_ORA_LEGALE == 1) & (u6.DATA == '2020-10-25')] = 96.80
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 3000) & (u6.FL_ORA_LEGALE == 1) & (u6.DATA == '2020-10-25')] = 97.35
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 4500) & (u6.FL_ORA_LEGALE == 1) & (u6.DATA == '2020-10-25')] = 97.35
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 10000) & (u6.FL_ORA_LEGALE == 1) & (u6.DATA == '2020-10-25')] = 97.35
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 11500) & (u6.FL_ORA_LEGALE == 1) & (u6.DATA == '2020-10-25')] = 96.80
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 13000) & (u6.FL_ORA_LEGALE == 1) & (u6.DATA == '2020-10-25')] = 96.25
u6['CONSUMO_ATTIVA_PRELEVATA'].loc[(u6.ORA == 14500) & (u6.FL_ORA_LEGALE == 1) & (u6.DATA == '2020-10-25')] = 95.70

In [38]:
u6.drop(list(range(98684,98780)), inplace=True)
#avevamo due tabelle dello stesso mese
#soluzione: tenere quella sistemata precedentemente e eliminare l'altra

In [39]:
len(u6[u6['DATA'] == '2020-10-25'])

96

Ora legale e ora solare.

In [40]:
u6[u6['DATA'] == '2018-10-28'].head(60)

POD       DATA     ORA  FL_ORA_LEGALE  \
28796  IT012E00491824 2018-10-28       0              2   
28797  IT012E00491824 2018-10-28    1500              2   
28798  IT012E00491824 2018-10-28    3000              2   
28799  IT012E00491824 2018-10-28    4500              2   
28800  IT012E00491824 2018-10-28   10000              2   
28801  IT012E00491824 2018-10-28   11500              2   
28802  IT012E00491824 2018-10-28   13000              2   
28803  IT012E00491824 2018-10-28   14500              2   
28804  IT012E00491824 2018-10-28   20000              2   
28805  IT012E00491824 2018-10-28   21500              2   
28806  IT012E00491824 2018-10-28   23000              2   
28807  IT012E00491824 2018-10-28   24500              2   
28808  IT012E00491824 2018-10-28   20000              1   
28809  IT012E00491824 2018-10-28   21500              1   
28810  IT012E00491824 2018-10-28   23000              1   
28811  IT012E00491824 2018-10-28   24500              1   
28812  IT012E00491824 2018-10-28   30000              1   
28813  IT012E00491824 2018-10-28   31500              1   
28814  IT012E00491824 2018-10-28   33000              1   
28815  IT012E00491824 2018-10-28   34500              1   
28816  IT012E00491824 2018-10-28   40000              1   
28817  IT012E00491824 2018-10-28   41500              1   
28818  IT012E00491824 2018-10-28   43000              1   
28819  IT012E00491824 2018-10-28   44500              1   
28820  IT012E00491824 2018-10-28   50000              1   
28821  IT012E00491824 2018-10-28   51500              1   
28822  IT012E00491824 2018-10-28   53000              1   
28823  IT012E00491824 2018-10-28   54500              1   
28824  IT012E00491824 2018-10-28   60000              1   
28825  IT012E00491824 2018-10-28   61500              1   
28826  IT012E00491824 2018-10-28   63000              1   
28827  IT012E00491824 2018-10-28   64500              1   
28828  IT012E00491824 2018-10-28   70000              1   
28829  IT012E00491824 2018-10-28   71500              1   
28830  IT012E00491824 2018-10-28   73000              1   
28831  IT012E00491824 2018-10-28   74500              1   
28832  IT012E00491824 2018-10-28   80000              1   
28833  IT012E00491824 2018-10-28   81500              1   
28834  IT012E00491824 2018-10-28   83000              1   
28835  IT012E00491824 2018-10-28   84500              1   
28836  IT012E00491824 2018-10-28   90000              1   
28837  IT012E00491824 2018-10-28   91500              1   
28838  IT012E00491824 2018-10-28   93000              1   
28839  IT012E00491824 2018-10-28   94500              1   
28840  IT012E00491824 2018-10-28  100000              1   
28841  IT012E00491824 2018-10-28  101500              1   
28842  IT012E00491824 2018-10-28  103000              1   
28843  IT012E00491824 2018-10-28  104500              1   
28844  IT012E00491824 2018-10-28  110000              1   
28845  IT012E00491824 2018-10-28  111500              1   
28846  IT012E00491824 2018-10-28  113000              1   
28847  IT012E00491824 2018-10-28  114500              1   
28848  IT012E00491824 2018-10-28  120000              1   
28849  IT012E00491824 2018-10-28  121500              1   
28850  IT012E00491824 2018-10-28  123000              1   
28851  IT012E00491824 2018-10-28  124500              1   
28852  IT012E00491824 2018-10-28  130000              1   
28853  IT012E00491824 2018-10-28  131500              1   
28854  IT012E00491824 2018-10-28  133000              1   
28855  IT012E00491824 2018-10-28  134500              1   

       CONSUMO_ATTIVA_PRELEVATA  CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA  \
28796                     60.50                                 27.50   
28797                     62.15                                 27.50   
28798                     63.80                                 29.15   
28799                     61.60                                 26.95   
28800                     60.50                      

In [41]:
u6.drop([28808, 28809, 28810, 28811], inplace=True)
#eliminare l'ora in più

In [42]:
u6[u6['DATA'] == '27-10-2019'].head(60)

POD       DATA     ORA  FL_ORA_LEGALE  \
63740  IT012E00491824 2019-10-27       0              2   
63741  IT012E00491824 2019-10-27    1500              2   
63742  IT012E00491824 2019-10-27    3000              2   
63743  IT012E00491824 2019-10-27    4500              2   
63744  IT012E00491824 2019-10-27   10000              2   
63745  IT012E00491824 2019-10-27   11500              2   
63746  IT012E00491824 2019-10-27   13000              2   
63747  IT012E00491824 2019-10-27   14500              2   
63748  IT012E00491824 2019-10-27   20000              2   
63749  IT012E00491824 2019-10-27   21500              2   
63750  IT012E00491824 2019-10-27   23000              2   
63751  IT012E00491824 2019-10-27   24500              2   
63752  IT012E00491824 2019-10-27   20000              1   
63753  IT012E00491824 2019-10-27   21500              1   
63754  IT012E00491824 2019-10-27   23000              1   
63755  IT012E00491824 2019-10-27   24500              1   
63756  IT012E00491824 2019-10-27   30000              1   
63757  IT012E00491824 2019-10-27   31500              1   
63758  IT012E00491824 2019-10-27   33000              1   
63759  IT012E00491824 2019-10-27   34500              1   
63760  IT012E00491824 2019-10-27   40000              1   
63761  IT012E00491824 2019-10-27   41500              1   
63762  IT012E00491824 2019-10-27   43000              1   
63763  IT012E00491824 2019-10-27   44500              1   
63764  IT012E00491824 2019-10-27   50000              1   
63765  IT012E00491824 2019-10-27   51500              1   
63766  IT012E00491824 2019-10-27   53000              1   
63767  IT012E00491824 2019-10-27   54500              1   
63768  IT012E00491824 2019-10-27   60000              1   
63769  IT012E00491824 2019-10-27   61500              1   
63770  IT012E00491824 2019-10-27   63000              1   
63771  IT012E00491824 2019-10-27   64500              1   
63772  IT012E00491824 2019-10-27   70000              1   
63773  IT012E00491824 2019-10-27   71500              1   
63774  IT012E00491824 2019-10-27   73000              1   
63775  IT012E00491824 2019-10-27   74500              1   
63776  IT012E00491824 2019-10-27   80000              1   
63777  IT012E00491824 2019-10-27   81500              1   
63778  IT012E00491824 2019-10-27   83000              1   
63779  IT012E00491824 2019-10-27   84500              1   
63780  IT012E00491824 2019-10-27   90000              1   
63781  IT012E00491824 2019-10-27   91500              1   
63782  IT012E00491824 2019-10-27   93000              1   
63783  IT012E00491824 2019-10-27   94500              1   
63784  IT012E00491824 2019-10-27  100000              1   
63785  IT012E00491824 2019-10-27  101500              1   
63786  IT012E00491824 2019-10-27  103000              1   
63787  IT012E00491824 2019-10-27  104500              1   
63788  IT012E00491824 2019-10-27  110000              1   
63789  IT012E00491824 2019-10-27  111500              1   
63790  IT012E00491824 2019-10-27  113000              1   
63791  IT012E00491824 2019-10-27  114500              1   
63792  IT012E00491824 2019-10-27  120000              1   
63793  IT012E00491824 2019-10-27  121500              1   
63794  IT012E00491824 2019-10-27  123000              1   
63795  IT012E00491824 2019-10-27  124500              1   
63796  IT012E00491824 2019-10-27  130000              1   
63797  IT012E00491824 2019-10-27  131500              1   
63798  IT012E00491824 2019-10-27  133000              1   
63799  IT012E00491824 2019-10-27  134500              1   

       CONSUMO_ATTIVA_PRELEVATA  CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA  \
63740                    115.50                                 45.65   
63741                    116.05                                 46.75   
63742                    115.50                                 46.20   
63743                    114.95                                 46.20   
63744                    114.95                      

In [43]:
u6.drop([63752, 63753, 63754, 63755], inplace=True)
#eliminare l'ora in più

In [44]:
u6[u6['DATA'] == '25-03-2018'].head(60)

POD       DATA     ORA  FL_ORA_LEGALE  \
7968  IT012E00491824 2018-03-25       0              1   
7969  IT012E00491824 2018-03-25    1500              1   
7970  IT012E00491824 2018-03-25    3000              1   
7971  IT012E00491824 2018-03-25    4500              1   
7972  IT012E00491824 2018-03-25   10000              1   
7973  IT012E00491824 2018-03-25   11500              1   
7974  IT012E00491824 2018-03-25   13000              1   
7975  IT012E00491824 2018-03-25   14500              1   
7976  IT012E00491824 2018-03-25   30000              2   
7977  IT012E00491824 2018-03-25   31500              2   
7978  IT012E00491824 2018-03-25   33000              2   
7979  IT012E00491824 2018-03-25   34500              2   
7980  IT012E00491824 2018-03-25   40000              2   
7981  IT012E00491824 2018-03-25   41500              2   
7982  IT012E00491824 2018-03-25   43000              2   
7983  IT012E00491824 2018-03-25   44500              2   
7984  IT012E00491824 2018-03-25   50000              2   
7985  IT012E00491824 2018-03-25   51500              2   
7986  IT012E00491824 2018-03-25   53000              2   
7987  IT012E00491824 2018-03-25   54500              2   
7988  IT012E00491824 2018-03-25   60000              2   
7989  IT012E00491824 2018-03-25   61500              2   
7990  IT012E00491824 2018-03-25   63000              2   
7991  IT012E00491824 2018-03-25   64500              2   
7992  IT012E00491824 2018-03-25   70000              2   
7993  IT012E00491824 2018-03-25   71500              2   
7994  IT012E00491824 2018-03-25   73000              2   
7995  IT012E00491824 2018-03-25   74500              2   
7996  IT012E00491824 2018-03-25   80000              2   
7997  IT012E00491824 2018-03-25   81500              2   
7998  IT012E00491824 2018-03-25   83000              2   
7999  IT012E00491824 2018-03-25   84500              2   
8000  IT012E00491824 2018-03-25   90000              2   
8001  IT012E00491824 2018-03-25   91500              2   
8002  IT012E00491824 2018-03-25   93000              2   
8003  IT012E00491824 2018-03-25   94500              2   
8004  IT012E00491824 2018-03-25  100000              2   
8005  IT012E00491824 2018-03-25  101500              2   
8006  IT012E00491824 2018-03-25  103000              2   
8007  IT012E00491824 2018-03-25  104500              2   
8008  IT012E00491824 2018-03-25  110000              2   
8009  IT012E00491824 2018-03-25  111500              2   
8010  IT012E00491824 2018-03-25  113000              2   
8011  IT012E00491824 2018-03-25  114500              2   
8012  IT012E00491824 2018-03-25  120000              2   
8013  IT012E00491824 2018-03-25  121500              2   
8014  IT012E00491824 2018-03-25  123000              2   
8015  IT012E00491824 2018-03-25  124500              2   
8016  IT012E00491824 2018-03-25  130000              2   
8017  IT012E00491824 2018-03-25  131500              2   
8018  IT012E00491824 2018-03-25  133000              2   
8019  IT012E00491824 2018-03-25  134500              2   
8020  IT012E00491824 2018-03-25  140000              2   
8021  IT012E00491824 2018-03-25  141500              2   
8022  IT012E00491824 2018-03-25  143000              2   
8023  IT012E00491824 2018-03-25  144500              2   
8024  IT012E00491824 2018-03-25  150000              2   
8025  IT012E00491824 2018-03-25  151500              2   
8026  IT012E00491824 2018-03-25  153000              2   
8027  IT012E00491824 2018-03-25  154500              2   

      CONSUMO_ATTIVA_PRELEVATA  CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA  \
7968                     81.40                                 39.60   
7969                     81.40                                 40.15   
7970                     81.40                                 39.60   
7971                     81.40                                 39.05   
7972                     81.40                                 39.05   
7973                     81.40                

In [45]:
primavera1 = {'DATA': ['20180325','20180325','20180325','20180325'], 'ORA': [20000,21500,23000,24500], 'CONSUMO_ATTIVA_PRELEVATA': [80.85,80.85,81.40,81.40]}
primavera1 = pd.DataFrame(data = primavera1, index = [7975.1, 7975.2, 7975.3, 7975.4])
primavera1['DATA'] = pd.to_datetime(primavera1['DATA'], format = '%Y%m%d')
u6 = u6.append(primavera1, ignore_index=False)
u6 = u6.sort_index().reset_index(drop=True)
#aggiungere l'ora con i valori della mezz'ora precedente e successiva

In [46]:
u6[u6['DATA'] == '2019-03-31'].head(60)

POD       DATA     ORA  FL_ORA_LEGALE  \
43584  IT012E00491824 2019-03-31       0            1.0   
43585  IT012E00491824 2019-03-31    1500            1.0   
43586  IT012E00491824 2019-03-31    3000            1.0   
43587  IT012E00491824 2019-03-31    4500            1.0   
43588  IT012E00491824 2019-03-31   10000            1.0   
43589  IT012E00491824 2019-03-31   11500            1.0   
43590  IT012E00491824 2019-03-31   13000            1.0   
43591  IT012E00491824 2019-03-31   14500            1.0   
43592  IT012E00491824 2019-03-31   30000            2.0   
43593  IT012E00491824 2019-03-31   31500            2.0   
43594  IT012E00491824 2019-03-31   33000            2.0   
43595  IT012E00491824 2019-03-31   34500            2.0   
43596  IT012E00491824 2019-03-31   40000            2.0   
43597  IT012E00491824 2019-03-31   41500            2.0   
43598  IT012E00491824 2019-03-31   43000            2.0   
43599  IT012E00491824 2019-03-31   44500            2.0   
43600  IT012E00491824 2019-03-31   50000            2.0   
43601  IT012E00491824 2019-03-31   51500            2.0   
43602  IT012E00491824 2019-03-31   53000            2.0   
43603  IT012E00491824 2019-03-31   54500            2.0   
43604  IT012E00491824 2019-03-31   60000            2.0   
43605  IT012E00491824 2019-03-31   61500            2.0   
43606  IT012E00491824 2019-03-31   63000            2.0   
43607  IT012E00491824 2019-03-31   64500            2.0   
43608  IT012E00491824 2019-03-31   70000            2.0   
43609  IT012E00491824 2019-03-31   71500            2.0   
43610  IT012E00491824 2019-03-31   73000            2.0   
43611  IT012E00491824 2019-03-31   74500            2.0   
43612  IT012E00491824 2019-03-31   80000            2.0   
43613  IT012E00491824 2019-03-31   81500            2.0   
43614  IT012E00491824 2019-03-31   83000            2.0   
43615  IT012E00491824 2019-03-31   84500            2.0   
43616  IT012E00491824 2019-03-31   90000            2.0   
43617  IT012E00491824 2019-03-31   91500            2.0   
43618  IT012E00491824 2019-03-31   93000            2.0   
43619  IT012E00491824 2019-03-31   94500            2.0   
43620  IT012E00491824 2019-03-31  100000            2.0   
43621  IT012E00491824 2019-03-31  101500            2.0   
43622  IT012E00491824 2019-03-31  103000            2.0   
43623  IT012E00491824 2019-03-31  104500            2.0   
43624  IT012E00491824 2019-03-31  110000            2.0   
43625  IT012E00491824 2019-03-31  111500            2.0   
43626  IT012E00491824 2019-03-31  113000            2.0   
43627  IT012E00491824 2019-03-31  114500            2.0   
43628  IT012E00491824 2019-03-31  120000            2.0   
43629  IT012E00491824 2019-03-31  121500            2.0   
43630  IT012E00491824 2019-03-31  123000            2.0   
43631  IT012E00491824 2019-03-31  124500            2.0   
43632  IT012E00491824 2019-03-31  130000            2.0   
43633  IT012E00491824 2019-03-31  131500            2.0   
43634  IT012E00491824 2019-03-31  133000            2.0   
43635  IT012E00491824 2019-03-31  134500            2.0   
43636  IT012E00491824 2019-03-31  140000            2.0   
43637  IT012E00491824 2019-03-31  141500            2.0   
43638  IT012E00491824 2019-03-31  143000            2.0   
43639  IT012E00491824 2019-03-31  144500            2.0   
43640  IT012E00491824 2019-03-31  150000            2.0   
43641  IT012E00491824 2019-03-31  151500            2.0   
43642  IT012E00491824 2019-03-31  153000            2.0   
43643  IT012E00491824 2019-03-31  154500            2.0   

       CONSUMO_ATTIVA_PRELEVATA  CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA  \
43584                    108.35                                 46.75   
43585                    107.80                                 46.20   
43586                    108.35                                 46.20   
43587                    107.80                                 46.20   
43588                    107.80                      

In [47]:
primavera2 = {'DATA': ['20190331','20190331','20190331','20190331'], 'ORA': [20000,21500,23000,24500], 'CONSUMO_ATTIVA_PRELEVATA': [107.80,107.80,107.25,107.25]}
primavera2 = pd.DataFrame(data = primavera2, index = [43595.1, 43595.2, 43595.3, 43595.4])
primavera2['DATA'] = pd.to_datetime(primavera2['DATA'], format = '%Y%m%d')
u6 = u6.append(primavera2, ignore_index=False)
u6 = u6.sort_index().reset_index(drop=True)
#aggiungere l'ora con i valori della mezz'ora precedente e successiva

In [48]:
u6[u6['DATA'] == '2020-03-29'].head(60)

POD       DATA     ORA  FL_ORA_LEGALE  \
78528  IT012E00491824 2020-03-29       0            1.0   
78529  IT012E00491824 2020-03-29    1500            1.0   
78530  IT012E00491824 2020-03-29    3000            1.0   
78531  IT012E00491824 2020-03-29    4500            1.0   
78532  IT012E00491824 2020-03-29   10000            1.0   
78533  IT012E00491824 2020-03-29   11500            1.0   
78534  IT012E00491824 2020-03-29   13000            1.0   
78535  IT012E00491824 2020-03-29   14500            1.0   
78536  IT012E00491824 2020-03-29   30000            2.0   
78537  IT012E00491824 2020-03-29   31500            2.0   
78538  IT012E00491824 2020-03-29   33000            2.0   
78539  IT012E00491824 2020-03-29   34500            2.0   
78540  IT012E00491824 2020-03-29   40000            2.0   
78541  IT012E00491824 2020-03-29   41500            2.0   
78542  IT012E00491824 2020-03-29   43000            2.0   
78543  IT012E00491824 2020-03-29   44500            2.0   
78544  IT012E00491824 2020-03-29   50000            2.0   
78545  IT012E00491824 2020-03-29   51500            2.0   
78546  IT012E00491824 2020-03-29   53000            2.0   
78547  IT012E00491824 2020-03-29   54500            2.0   
78548  IT012E00491824 2020-03-29   60000            2.0   
78549  IT012E00491824 2020-03-29   61500            2.0   
78550  IT012E00491824 2020-03-29   63000            2.0   
78551  IT012E00491824 2020-03-29   64500            2.0   
78552  IT012E00491824 2020-03-29   70000            2.0   
78553  IT012E00491824 2020-03-29   71500            2.0   
78554  IT012E00491824 2020-03-29   73000            2.0   
78555  IT012E00491824 2020-03-29   74500            2.0   
78556  IT012E00491824 2020-03-29   80000            2.0   
78557  IT012E00491824 2020-03-29   81500            2.0   
78558  IT012E00491824 2020-03-29   83000            2.0   
78559  IT012E00491824 2020-03-29   84500            2.0   
78560  IT012E00491824 2020-03-29   90000            2.0   
78561  IT012E00491824 2020-03-29   91500            2.0   
78562  IT012E00491824 2020-03-29   93000            2.0   
78563  IT012E00491824 2020-03-29   94500            2.0   
78564  IT012E00491824 2020-03-29  100000            2.0   
78565  IT012E00491824 2020-03-29  101500            2.0   
78566  IT012E00491824 2020-03-29  103000            2.0   
78567  IT012E00491824 2020-03-29  104500            2.0   
78568  IT012E00491824 2020-03-29  110000            2.0   
78569  IT012E00491824 2020-03-29  111500            2.0   
78570  IT012E00491824 2020-03-29  113000            2.0   
78571  IT012E00491824 2020-03-29  114500            2.0   
78572  IT012E00491824 2020-03-29  120000            2.0   
78573  IT012E00491824 2020-03-29  121500            2.0   
78574  IT012E00491824 2020-03-29  123000            2.0   
78575  IT012E00491824 2020-03-29  124500            2.0   
78576  IT012E00491824 2020-03-29  130000            2.0   
78577  IT012E00491824 2020-03-29  131500            2.0   
78578  IT012E00491824 2020-03-29  133000            2.0   
78579  IT012E00491824 2020-03-29  134500            2.0   
78580  IT012E00491824 2020-03-29  140000            2.0   
78581  IT012E00491824 2020-03-29  141500            2.0   
78582  IT012E00491824 2020-03-29  143000            2.0   
78583  IT012E00491824 2020-03-29  144500            2.0   
78584  IT012E00491824 2020-03-29  150000            2.0   
78585  IT012E00491824 2020-03-29  151500            2.0   
78586  IT012E00491824 2020-03-29  153000            2.0   
78587  IT012E00491824 2020-03-29  154500            2.0   

       CONSUMO_ATTIVA_PRELEVATA  CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA  \
78528                    106.70                                 41.80   
78529                    107.80                                 42.35   
78530                    107.25                                 42.35   
78531                    106.70                                 41.80   
78532                    107.25                      

In [49]:
primavera3 = {'DATA': ['20200329','20200329','20200329','20200329'], 'ORA': [20000,21500,23000,24500], 'CONSUMO_ATTIVA_PRELEVATA': [106.70,106.70,106.70,106.70]}
primavera3 = pd.DataFrame(data = primavera3, index = [78535.1, 78535.2, 78535.3, 78535.4])
primavera3['DATA'] = pd.to_datetime(primavera3['DATA'], format = '%Y%m%d')
u6 = u6.append(primavera3, ignore_index=False)
u6 = u6.sort_index().reset_index(drop=True)
#aggiungere l'ora con i valori della mezz'ora precedente e successiva

Ultimo controllo generale: zeri e 96 righe per giorno.

In [50]:
u6[u6['CONSUMO_ATTIVA_PRELEVATA'] == 0]

Empty DataFrame
Columns: [POD, DATA, ORA, FL_ORA_LEGALE, CONSUMO_ATTIVA_PRELEVATA, CONSUMO_REATTIVA_INDUTTIVA_PRELEVATA, POTENZA_MASSIMA, TIPO_DATO, Unnamed: 1, ORA.1, POTENZA ATTIVA KW, Unnamed: 11, Unnamed: 12, Unnamed: 13, Unnamed: 14, Unnamed: 15, Unnamed: 16, Unnamed: 17, Unnamed: 18, Unnamed: 19, Unnamed: 20, Unnamed: 21, Unnamed: 22, Unnamed: 23, Unnamed: 24, Unnamed: 25, Unnamed: 26, Unnamed: 27, Unnamed: 28, anno, mese, giorno, giorno/sett, ora, Unnamed: 8, Unnamed: 9, Unnamed: 10]
Index: []

[0 rows x 37 columns]

In [51]:
giorni = u6['DATA'].unique()
for giorno in giorni :
    dati_del_giorno = u6[u6['DATA'] == giorno]
    if len(dati_del_giorno) != 96:
        print(giorno, len(dati_del_giorno))
#tutti i giorni hanno 96 righe

Selezione delle variabili di interesse e conversione in KWH.

In [52]:
u6=u6[['DATA', 'ORA', 'CONSUMO_ATTIVA_PRELEVATA']]
u6['Consumo15'] = u6['CONSUMO_ATTIVA_PRELEVATA']*15
#consumo15 = consumo complessivo in KW durante il quarto d'ora

In [53]:
def tempo(ora) :
    if ora == 0 or ora == 1500 or ora == 3000 or ora == 4500 : 
        return 0
    else :
        return (ora//10000)   

In [54]:
u6['Tempo'] = u6['ORA'].apply(tempo)

In [55]:
u6_agg_ore = pd.DataFrame(u6[['DATA', 'Tempo', 'Consumo15']].groupby(['DATA', 'Tempo']).sum()['Consumo15']).reset_index()
u6_agg_ore.rename(columns={'Tempo': 'ORA', 'Consumo15' : 'CONSUMO_KWH_U6'}, inplace = True)
#conversione da KW a KWH

In [56]:
#u6_agg_ore.to_excel(r'C:\Users\Nabil-PC\Downloads\Progetto Lab\u6_agg_ore.xlsx', index = False)
#compreso il mese errato di giugno 2020 per successive analisi